In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTForImageClassification
import os
import time
from tqdm import tqdm  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = "/kaggle/input/fer2013"  
num_classes = 7  
batch_size = 32


transform = transforms.Compose([
    transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),  
    transforms.CenterCrop(224),  # Recorte central
    transforms.ToTensor(),  # Converter para tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])


datasets_dict = {
    "train": datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform),
    "val": datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform)  
}

dataloaders = {
    phase: DataLoader(datasets_dict[phase], batch_size=batch_size, shuffle=(phase == "train"), num_workers=4)
    for phase in ["train", "val"]
}

data_sizes = {phase: len(datasets_dict[phase]) for phase in ["train", "val"]}
print(f"Tamanho do conjunto de treinamento: {data_sizes['train']}")
print(f"Tamanho do conjunto de validação: {data_sizes['val']}")

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",  
    num_labels=num_classes  
)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)

def train_and_validate(model, dataloaders, criterion, optimizer, num_epochs=10, checkpoint_dir="/kaggle/working/checkpoints"):
    os.makedirs(checkpoint_dir, exist_ok=True)  
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    
    for epoch in range(num_epochs):
        print(f"\epoca {epoch + 1}/{num_epochs}")
        print("-" * 40)
        epoch_start = time.time()
        
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            progress_bar = tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Processando")
            
            for inputs, labels in progress_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs).logits
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                progress_bar.set_postfix(loss=loss.item())
            
            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_corrects.double() / data_sizes[phase]
            history[f"{phase}_loss"].append(epoch_loss)
            history[f"{phase}_acc"].append(epoch_acc)
            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        
        epoch_end = time.time()
        print(f"Tempo de execução da epocj levou: {epoch_end - epoch_start:.2f} segundos")
        
        if (epoch + 1) % 5 == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f"vit_epoch_{epoch + 1}.pt")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Checkpoint salvo em: {checkpoint_path}")
    
    print("\nTreinamento concluído!")
    return history

history = train_and_validate(model, dataloaders, criterion, optimizer, num_epochs=10)
output_dir = "/kaggle/working/vit-fine-tuned-facial-expressions"
os.makedirs(output_dir, exist_ok=True)  # Garantir que o diretório exista
model.save_pretrained(output_dir)
print(f"Modelo salvo com sucesso em: {output_dir}")


Usando dispositivo: cuda
Tamanho do conjunto de treinamento: 28709
Tamanho do conjunto de validação: 7178


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Época 1/10
----------------------------------------


...

Val Loss: 1.2809 Acc: 0.6977
Tempo de execução da época: 1043.74 segundos

Época 10/10
----------------------------------------


Train Processando: 100%|██████████| 898/898 [16:02<00:00,  1.07s/it, loss=0.546]  


Train Loss: 0.1005 Acc: 0.9656


Val Processando: 100%|██████████| 225/225 [01:21<00:00,  2.76it/s, loss=0.559]  


Val Loss: 1.3895 Acc: 0.6806
Tempo de execução da época: 1043.87 segundos
Checkpoint salvo em: /kaggle/working/checkpoints/vit_epoch_10.pt

Treinamento concluído!
Modelo salvo com sucesso em: /kaggle/working/vit-fine-tuned-facial-expressions


In [3]:
import matplotlib.pyplot as plt
def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Converter tensores para CPU e depois para numpy (se necessário)
    train_loss = history['train_loss'].cpu().numpy() if isinstance(history['train_loss'], torch.Tensor) else history['train_loss']
    val_loss = history['val_loss'].cpu().numpy() if isinstance(history['val_loss'], torch.Tensor) else history['val_loss']
    train_acc = history['train_acc'].cpu().numpy() if isinstance(history['train_acc'], torch.Tensor) else history['train_acc']
    val_acc = history['val_acc'].cpu().numpy() if isinstance(history['val_acc'], torch.Tensor) else history['val_acc']
    
    plt.figure(figsize=(12, 5))
    
    # Plot da Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label='Treino - Loss', marker='o')
    plt.plot(epochs, val_loss, label='Validação - Loss', marker='o')
    plt.title('Loss ao longo das épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot da Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label='Treino - Accuracy', marker='o')
    plt.plot(epochs, val_acc, label='Validação - Accuracy', marker='o')
    plt.title('Accuracy ao longo das épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plotar o histórico de treinamento
plot_training_history(history)

NameError: name 'history' is not defined